In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    ResNet50, VGG16, InceptionV3, MobileNetV2, EfficientNetB0
)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2025-07-08 07:14:59.098777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751958899.469647      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751958899.568145      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_dir = '/kaggle/input/teeth-dataset-0/Teeth_Dataset/Training'
val_dir = '/kaggle/input/teeth-dataset-0/Teeth_Dataset/Validation'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
preprocessing_map = {
    "ResNet50": resnet_preprocess,
    "VGG16": vgg_preprocess,
    "InceptionV3": inception_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess
}

In [ ]:
base_models = {
    "ResNet50": ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "InceptionV3": InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "MobileNetV2": MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "EfficientNetB0": EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}


I0000 00:00:1751958989.671011      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751958989.671749      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
results = {}

for name, base_model in base_models.items():
    print(f"\n🚀 Training {name}...")


    train_datagen = ImageDataGenerator(preprocessing_function=preprocessing_map[name])
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_map[name])

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )


    base_model.trainable = False

    # Add custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(train_generator.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f"{name}_best.h5", save_best_only=True)
    ]

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=25,
        callbacks=callbacks
    )

    val_acc = max(history.history['val_accuracy'])
    results[name] = val_acc



🚀 Training ResNet50...
Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


I0000 00:00:1751959016.738424      98 service.cc:148] XLA service 0x7ba46806d260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751959016.739734      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751959016.739753      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751959018.295014      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


 2/97 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.1016 - loss: 3.8031  

I0000 00:00:1751959023.983382      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 406ms/step - accuracy: 0.4228 - loss: 1.9431 - val_accuracy: 0.7695 - val_loss: 0.7051
Epoch 2/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 144ms/step - accuracy: 0.7337 - loss: 0.7109 - val_accuracy: 0.8628 - val_loss: 0.4657
Epoch 3/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 142ms/step - accuracy: 0.8206 - loss: 0.5000 - val_accuracy: 0.9047 - val_loss: 0.3155
Epoch 4/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 141ms/step - accuracy: 0.8835 - loss: 0.3337 - val_accuracy: 0.9339 - val_loss: 0.2395
Epoch 5/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 15s 150ms/step - accuracy: 0.9171 - loss: 0.2488 - val_accuracy: 0.9339 - val_loss: 0.2171
Epoch 6/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - accuracy: 0.9267 - loss: 0.2049 - val_accuracy: 0.9416 - val_loss: 0.1895
Epoch 7/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 145ms/step - accuracy: 0.9286 - loss: 0.1898 - val_accuracy: 0.9543 - val_loss: 0.1542
Epoch 8/25
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 140ms/step - accuracy: 0.9650 - loss: 0.1240 - val_accuracy: 0.978

In [ ]:
print("\n✅ Results for all models:")
print(results)


best_model_name = max(results, key=results.get)
print(f"\n🏆 Best Model: {best_model_name} with Val Acc: {results[best_model_name]:.4f}")


✅ Results for all models:
{'ResNet50': 0.9834630489349365, 'VGG16': 0.9863813519477844, 'InceptionV3': 0.8861867785453796, 'MobileNetV2': 0.966926097869873, 'EfficientNetB0': 0.9931906461715698}

🏆 Best Model: EfficientNetB0 with Val Acc: 0.9932


In [ ]:
model.save('saved_model.h5')


In [ ]:
test_dir = '/kaggle/input/teeth-dataset-0/Teeth_Dataset/Testing'

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']  # ✅ هنا بتحدد بس الكلاسات المطلوبة
)


Found 1028 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('saved_model.h5', compile=False)

In [ ]:
best_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🔹 Evaluate
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 256ms/step - accuracy: 0.9860 - loss: 0.0364
✅ Test Accuracy: 0.9874


In [ ]:
!pip install streamlit pyngrok

In [ ]:
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# عنوان التطبيق
st.title("Teeth Classification App")

# ارفع الصورة
uploaded_file = st.file_uploader("Upload a teeth image", type=["jpg", "png"])

if uploaded_file is not None:
    # اعرض الصورة
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)

    # جهّز الصورة
    img = image.resize((224, 224))  # غيّر المقاس حسب موديلك
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    # حمّل الموديل
    model = load_model('saved_model.h5')

    # التوقع
    prediction = model.predict(img_array)

    st.write(f"Prediction: {prediction}")
""")


In [ ]:
from google.colab import files
files.upload()


Saving saved_model.h5 to saved_model.h5
Buffered data was truncated after reaching the output size limit.

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('saved_model.keras')
